Preparer les donnees camera sous format csv en fusionnat entre et sortie.

Ce code marche pour P1 , P6, P10 , P14, P18, P21 et P25 en adaptant les colonnes pour chaque fichier. et des fois le nom de la conne horodate.

La colonne sens indique si c'est entrée vers le campus universitaire(sens=0) ou sortie(sens=1).


In [27]:
import pandas as pd

# Charger le fichier Excel, en spécifiant que ca commence a la ligne 2 !!!
df = pd.read_excel('./P25.xls', sheet_name='Données_horaire', header=1)

df.dropna(how='all', inplace=True)
df = df[~df['Horodate'].str.contains('total', case=False, na=False)]

df['Datetime'] = pd.to_datetime(df['Horodate'])
df['DayOfWeek'] = df['Datetime'].dt.dayofweek

columns = ['VL', 'UT', 'EDPM', 'VELO', 'MOTO', 'PL_1', 'PL_2', 'BUS']

# Créer un DataFrame pour l'Entrée (colonnes B à I)
entree = df.iloc[:, 1:9].copy()
entree.columns = columns  
entree['Sens'] = 0  
entree['Datetime'] = df['Datetime']  
entree['DayOfWeek'] = df['DayOfWeek'] 

# Créer un DataFrame pour la Sortie (colonnes K à R)
sortie = df.iloc[:, 10:18].copy()
sortie.columns = columns  
sortie['Sens'] = 1  
sortie['Datetime'] = df['Datetime']  
sortie['DayOfWeek'] = df['DayOfWeek'] 

df_final = pd.concat([entree, sortie], ignore_index=True)

for col in columns:
    df_final[col] = df_final[col].astype('Int64')  # 'Int64' (capital 'I') !!!

df_final = df_final[['Datetime', 'DayOfWeek',  'Sens','VELO', 'MOTO', 'VL', 'UT', 'EDPM', 'PL_1', 'PL_2', 'BUS']]


df_final.sort_values(by=['Datetime'], inplace=True)

df_final.to_csv('./P25.csv', index=False)



Pour les postes P7 ( ayant que 6 types de vehicules)

In [ ]:
import pandas as pd

# Charger le fichier Excel, en spécifiant que les en-têtes sont sur la deuxième ligne
df = pd.read_excel('./P7.xls', sheet_name='Données_horaires', header=1)
# Supprimer les lignes vides
df.dropna(how='all', inplace=True)

# Supprimer les lignes contenant 'total'
df = df[~df['Horodate'].str.contains('total', case=False, na=False)]

df['Datetime'] = pd.to_datetime(df['Horodate'])
df['DayOfWeek'] = df['Datetime'].dt.dayofweek

columns = ['VL', 'VELO', 'UT', 'MOTO', 'EDPM', 'PL_1']

entree = df.iloc[:, 1:7].copy()
entree.columns = columns  # Renommer les colonnes pour correspondre
entree['Sens'] = 0  # 0 pour Entrée
entree['Datetime'] = df['Datetime']  # Ajouter la colonne Datetime
entree['DayOfWeek'] = df['DayOfWeek'] 


sortie = df.iloc[:, 8:14].copy()
sortie.columns = columns  # Assurer la correspondance des noms de colonnes
sortie['Sens'] = 1  # 1 pour Sortie
sortie['Datetime'] = df['Datetime']  # Ajouter la colonne Datetime
sortie['DayOfWeek'] = df['DayOfWeek'] 

df_final = pd.concat([entree, sortie], ignore_index=True)


for col in columns:
    df_final[col] = df_final[col].astype('Int64')  # 'Int64' (capital 'I') !!!

df_final = df_final[['Datetime', 'DayOfWeek',  'Sens','VELO', 'MOTO', 'VL', 'UT', 'EDPM', 'PL_1']]

# Trier les données par 'Datetime'
df_final.sort_values(by=['Datetime'], inplace=True)

# Exporter en CSV
df_final.to_csv('./P7.csv', index=False)


Ensuite pour le poste 8, on posséde plusieurs trajectoires.

La direction entrée pour S1 n'est pas une entrée vers le campus mais plutôt une sortie donc pour les 3 feuilles S1, S2 et S3 faudra changer le sens et mettre le contraire.

De même pour le poste 20.

In [24]:
import pandas as pd

df = pd.read_excel('./P20.xls', sheet_name='Données_horaire_S3', header=1)


df.dropna(how='all', inplace=True)

df = df[~df['horodate'].str.contains('total', case=False, na=False)]

df['Datetime'] = pd.to_datetime(df['horodate'])
df['DayOfWeek'] = df['Datetime'].dt.dayofweek


columns = ['VL', 'VELO', 'UT', 'MOTO', 'EDPM', 'PL_1', 'PL_2', 'BUS']

# Créer un DataFrame pour l'Entrée (colonnes B à I)
entree = df.iloc[:, 1:9].copy()
entree.columns = columns 
entree['Sens'] = 1  # 0 pour entre
entree['Datetime'] = df['Datetime']  
entree['DayOfWeek'] = df['DayOfWeek'] 

# Créer un DataFrame pour la Sortie (colonnes K à R)
sortie = df.iloc[:, 10:18].copy() 
sortie.columns = columns 
sortie['Sens'] = 0
sortie['Datetime'] = df['Datetime'] 
sortie['DayOfWeek'] = df['DayOfWeek'] 

# Concaténer les DataFrames d'Entrée et de Sortie
df_final = pd.concat([entree, sortie], ignore_index=True)

for col in columns:
    df_final[col] = df_final[col].astype('Int64')  # 'Int64' (capital 'I') !!!

df_final = df_final[['Datetime', 'DayOfWeek',  'Sens','VELO', 'MOTO', 'VL', 'UT', 'EDPM', 'PL_1', 'PL_2', 'BUS']]

df_final.sort_values(by=['Datetime'], inplace=True)

df_final.to_csv('./P20_S3.csv', index=False)


Ensuite faudra remplir les valeurs manquant pour les postes 12 et 13 .

Je commence par créer un fichier contenant toute la plage temporelle avec du vide pour les plages non presentes auparavant et en gardant le vide si existe deja.

In [11]:
import pandas as pd

# Charger le fichier Excel
df = pd.read_excel('./P13.xls', sheet_name='Données_horaire', header=1)

# Supprimer les lignes vides
df.dropna(how='all', inplace=True)

# Supprimer les lignes contenant 'total'
df = df[~df['Horodate'].str.contains('total', case=False, na=False)]

# Convertir 'Horodate' en datetime
df['Datetime'] = pd.to_datetime(df['Horodate'])
df.drop(['Horodate'], axis=1, inplace=True)  # Suppression de l'ancienne colonne 'Horodate'

# Générer une série temporelle complète entre les dates min et max
date_range = pd.date_range(start=df['Datetime'].min(), end=df['Datetime'].max(), freq='H')
date_range = pd.DataFrame(date_range, columns=['Datetime'])

# Ajouter la colonne du jour de la semaine pour la série temporelle complète
date_range['DayOfWeek'] = date_range['Datetime'].dt.dayofweek

df_full = pd.merge(date_range, df, on='Datetime', how='left')

df_full.sort_values('Datetime', inplace=True)

df_full.to_excel('P13_ordo.xlsx', index=False)



C:\Users\Admin\AppData\Local\Temp\ipykernel_8712\849327802.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=df['Datetime'].min(), end=df['Datetime'].max(), freq='H')


Ensuite pour remplir les valeurs manquantes :
1er solution abordée interpolation spline cubique.

2eme solution interpolation spline cubique en prenant en compte les jours de la semaines. doesn t work

interpolation par jour et par heure: doesn t work.

interpolation par heure : work but pas precis.

Interpolation spline heure et weekend : doesn t work.

Interpolation lineaire heure et weekend : work avec des donnes manquantes.


In [12]:
import pandas as pd
import numpy as np

df = pd.read_excel('./P13_ordo.xlsx')

df['Datetime'] = pd.to_datetime(df['Datetime'])
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek


columns_in = ['VL', 'UT', 'MOTO', 'VELO', 'EDPM', 'PL_1', 'PL_2', 'BUS']
columns_out = ['VL.1', 'UT.1', 'MOTO.1', 'VELO.1', 'EDPM.1', 'PL_1.1', 'PL_2.1', 'BUS.1']
all_columns = columns_in + columns_out

mean_values = df.groupby(['DayOfWeek', 'Hour'])[all_columns].transform('mean')
df[all_columns] = df[all_columns].fillna(mean_values)

df[all_columns] = df[all_columns].apply(lambda x: np.clip(x, 0, None))


df[all_columns] = df[all_columns].round().astype('Int64')

missing_after_fillna = df[all_columns].isna().sum()
print("Valeurs manquantes après fillna :", missing_after_fillna)

df.to_excel('P13_mean_HourDay.xlsx', index=False)


Valeurs manquantes après fillna : VL        0
UT        0
MOTO      0
VELO      0
EDPM      0
PL_1      0
PL_2      0
BUS       0
VL.1      0
UT.1      0
MOTO.1    0
VELO.1    0
EDPM.1    0
PL_1.1    0
PL_2.1    0
BUS.1     0
dtype: int64


Ici j'essaie avec la moyenne selon l heure et le weekend:worked.

Ici j'essaie avec la mediane selon l heure et le weekend:worked.

j applique la moyenne et la mediane sur le resultat de l interpolation lineaire par weekend et hour. 

In [8]:

import pandas as pd
import numpy as np

# Charger le fichier Excel
df = pd.read_excel('./P12_IL_HourWeekend.xlsx')

columns_in = ['VL', 'UT', 'VELO', 'MOTO', 'PL_1', 'PL_2', 'EDPM', 'BUS']
columns_out = ['VL.1', 'UT.1', 'VELO.1', 'MOTO.1', 'PL_1.1', 'PL_2.1', 'EDPM.1', 'BUS.1']
all_columns = columns_in + columns_out

grouped = df.groupby(['Weekend', 'Hour'])[all_columns].transform('mean')

df[all_columns] = df[all_columns].combine_first(grouped)
df[all_columns] = df[all_columns].apply(lambda x: np.clip(x, 0, None))

df[all_columns] = df[all_columns].round().astype('Int64')

df.to_excel('P12_mean_Il_WH.xlsx', index=False)


Après remplissage de données manquantes: il faut les mettre sous une forme csv comme les autres fichiers:

In [14]:

import pandas as pd
df = pd.read_excel('./P13_mean_HourDay.xlsx')

df.dropna(how='all', inplace=True)

# columns_in = ['VL', 'UT', 'VELO', 'MOTO', 'PL_1', 'PL_2', 'EDPM', 'BUS']
# columns_out = ['VL.1', 'UT.1', 'VELO.1', 'MOTO.1', 'PL_1.1', 'PL_2.1', 'EDPM.1', 'BUS.1']
columns_in = ['VL', 'UT', 'MOTO', 'VELO', 'EDPM', 'PL_1', 'PL_2', 'BUS']
columns_out = ['VL.1', 'UT.1', 'MOTO.1', 'VELO.1', 'EDPM.1', 'PL_1.1', 'PL_2.1', 'BUS.1']


entree = df[['Datetime', 'DayOfWeek', 'Hour'] + columns_in].copy()
entree.columns = ['Datetime', 'DayOfWeek', 'Hour'] + columns_in
entree['Sens'] = 0  # 0 pour Entrée

sortie = df[['Datetime', 'DayOfWeek', 'Hour'] + columns_out].copy()
sortie.columns = ['Datetime', 'DayOfWeek', 'Hour'] + columns_in
sortie['Sens'] = 1  # 1 pour Sortie

df_final = pd.concat([entree, sortie], ignore_index=True)

for col in columns_in:
    df_final[col] = df_final[col].astype('Int64')

df_final = df_final[['Datetime', 'DayOfWeek', 'Sens','VELO', 'MOTO', 'VL', 'UT', 'EDPM', 'PL_1', 'PL_2', 'BUS']]

df_final.sort_values(by=['Datetime'], inplace=True)

df_final.to_csv('./P13_mean_WH.csv', index=False)


Ici je compte à m'assurer que tous les plages horaires sont presentes sur le fichier 

In [26]:
import pandas as pd

df = pd.read_csv('./P25.csv')

df['Datetime'] = pd.to_datetime(df['Datetime'])
date_range = pd.date_range(start=df['Datetime'].min(), end=df['Datetime'].max(), freq='H')

full_range_df = pd.DataFrame([(date, sens) for date in date_range for sens in df['Sens'].unique()], columns=['Datetime', 'Sens'])

df_full = pd.merge(full_range_df, df, on=['Datetime', 'Sens'], how='left')

vehicle_columns = ['VELO', 'MOTO', 'VL', 'UT', 'EDPM', 'PL_1', 'PL_2', 'BUS']
df_full[vehicle_columns] = df_full[vehicle_columns].fillna(0).astype(int)

df_full['DayOfWeek'] = df_full['Datetime'].dt.dayofweek

df_full = df_full[['Datetime', 'DayOfWeek', 'Sens'] + vehicle_columns]
df_full.to_csv('./P25complet.csv', index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_21432\452281539.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=df['Datetime'].min(), end=df['Datetime'].max(), freq='H')


Finalement je compare les valeurs du poste 8

In [33]:
import pandas as pd

f1 = pd.read_csv('./P8_Ecomplet.csv')
f2 = pd.read_csv('./P8_S1complet.csv')
f3 = pd.read_csv('./P8_S2complet.csv')
f4 = pd.read_csv('./P8_S3complet.csv')

columns_to_compare = ['VELO', 'MOTO', 'VL', 'UT', 'EDPM', 'PL_1', 'PL_2', 'BUS']

def compare_files(df1, df2, df3, df4, columns):
    differences = []
    for _, row in df1.iterrows():
        datetime = row['Datetime']
        sens = row['Sens']
        row_differences = row.copy()
        has_difference = False
        for col in columns:
            value1 = row[col]
            value2 = df2[(df2['Datetime'] == datetime) & (df2['Sens'] == sens)][col].values
            value3 = df3[(df3['Datetime'] == datetime) & (df3['Sens'] == sens)][col].values
            value4 = df4[(df4['Datetime'] == datetime) & (df4['Sens'] == sens)][col].values
            
            if len(value2) > 0 and len(value3) > 0 and len(value4) > 0:
                difference = value1 - value2[0] - value3[0] - value4[0]
                row_differences[col] = difference
                if difference != 0:
                    has_difference = True
            else:
                print(f'Donnée manquante pour {datetime} - {col}')
                row_differences[col] = value1
        if has_difference:
            differences.append(row_differences)
    return pd.DataFrame(differences)

differences = compare_files(f1, f2, f3, f4, columns_to_compare)

for index, row in differences.iterrows():
    print(f"Datetime: {row['Datetime']}, Sens: {row['Sens']}, Differences: {row[columns_to_compare].to_dict()}")

pd.DataFrame(differences).to_csv('./P8_differences.csv', index=False)


Datetime: 2022-10-07 14:00:00, Sens: 0, Differences: {'VELO': -1, 'MOTO': 0, 'VL': -11, 'UT': -1, 'EDPM': 0, 'PL_1': 0, 'PL_2': 0, 'BUS': 0}
Datetime: 2022-10-07 14:00:00, Sens: 1, Differences: {'VELO': -1, 'MOTO': 0, 'VL': -11, 'UT': -1, 'EDPM': 0, 'PL_1': 0, 'PL_2': 0, 'BUS': 0}
Datetime: 2022-10-07 15:00:00, Sens: 0, Differences: {'VELO': -1, 'MOTO': 0, 'VL': -10, 'UT': -2, 'EDPM': -1, 'PL_1': 0, 'PL_2': 0, 'BUS': 0}
Datetime: 2022-10-07 15:00:00, Sens: 1, Differences: {'VELO': -1, 'MOTO': 0, 'VL': -10, 'UT': -2, 'EDPM': -1, 'PL_1': 0, 'PL_2': 0, 'BUS': 0}
Datetime: 2022-10-07 16:00:00, Sens: 0, Differences: {'VELO': -1, 'MOTO': 0, 'VL': -23, 'UT': -2, 'EDPM': -1, 'PL_1': 0, 'PL_2': 0, 'BUS': 0}
Datetime: 2022-10-07 16:00:00, Sens: 1, Differences: {'VELO': -1, 'MOTO': 0, 'VL': -23, 'UT': -2, 'EDPM': -1, 'PL_1': 0, 'PL_2': 0, 'BUS': 0}
Datetime: 2022-10-07 17:00:00, Sens: 0, Differences: {'VELO': -5, 'MOTO': 0, 'VL': -19, 'UT': 0, 'EDPM': 1, 'PL_1': 0, 'PL_2': 0, 'BUS': 0}
Datetime: 

je vais essayer de sommer en esperant que ca sera la differnece d'heure par exemple il depasse le capteur vers x:59 et doonc le 2eme capteur va le compter pour la prochaine heure.

In [35]:
df = pd.read_csv('./P8_differences.csv')
columns_to_sum = ['VELO', 'MOTO', 'VL', 'UT', 'EDPM', 'PL_1', 'PL_2', 'BUS']

sum_values = df[columns_to_sum].sum()
print("Sum of values for each column:")
print(sum_values)

Sum of values for each column:
VELO   -1204
MOTO     -70
VL     -5694
UT      -464
EDPM     -72
PL_1    -324
PL_2      -2
BUS        0
dtype: int64


Pour le poste 20

In [37]:
f1 = pd.read_csv('./P20_Ecomplet.csv')
f2 = pd.read_csv('./P20_S1complet.csv')
f3 = pd.read_csv('./P20_S2complet.csv')
f4 = pd.read_csv('./P20_S3complet.csv')

columns_to_compare = ['VELO', 'MOTO', 'VL', 'UT', 'EDPM', 'PL_1', 'PL_2', 'BUS']

def compare_files(df1, df2, df3, df4, columns):
    differences = []
    for _, row in df1.iterrows():
        datetime = row['Datetime']
        sens = row['Sens']
        row_differences = row.copy()
        has_difference = False
        for col in columns:
            value1 = row[col]
            value2 = df2[(df2['Datetime'] == datetime) & (df2['Sens'] == sens)][col].values
            value3 = df3[(df3['Datetime'] == datetime) & (df3['Sens'] == sens)][col].values
            value4 = df4[(df4['Datetime'] == datetime) & (df4['Sens'] == sens)][col].values
            
            if len(value2) > 0 and len(value3) > 0 and len(value4) > 0:
                difference = value1 - value2[0] - value3[0] - value4[0]
                row_differences[col] = difference
                if difference != 0:
                    has_difference = True
            else:
                print(f'Donnée manquante pour {datetime} - {col}')
                row_differences[col] = value1
        if has_difference:
            differences.append(row_differences)
    return pd.DataFrame(differences)

differences = compare_files(f1, f2, f3, f4, columns_to_compare)


In [38]:
df = pd.read_csv('./P20_differences.csv')
columns_to_sum = ['VELO', 'MOTO', 'VL', 'UT', 'EDPM', 'PL_1', 'PL_2', 'BUS']

sum_values = df[columns_to_sum].sum()
print("Sum of values for each column:")
print(sum_values)

Sum of values for each column:
VELO     -2
MOTO    -38
VL     -376
UT     -126
EDPM      0
PL_1    -82
PL_2     -2
BUS       0
dtype: int64
